# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [22]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [23]:
clean_city_data = pd.read_csv("../WeatherPy/cleancitydata.csv")
clean_city_data


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Rikitea,-23.1203,-134.9692,74.91,79,100,19.22,PF,1635043325
1,Butaritari,3.0707,172.7902,81.21,72,40,7.81,KI,1635043326
2,New Norfolk,-42.7826,147.0587,52.92,60,100,3.00,AU,1635043326
3,Ust-Tsilma,65.4410,52.1498,22.80,98,84,3.71,RU,1635043326
4,Kosonsoy,41.2561,71.5508,34.36,38,0,1.16,UZ,1635043327
...,...,...,...,...,...,...,...,...,...
527,Verkhnyaya Inta,65.9811,60.3094,21.79,96,77,13.44,RU,1635043478
528,Nome,64.5011,-165.4064,32.07,74,1,4.61,US,1635043478
529,Acarí,-15.4311,-74.6158,65.46,70,41,3.94,PE,1635043479
530,Alice Springs,-23.7000,133.8833,78.35,21,7,17.27,AU,1635043479


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [24]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = clean_city_data[["Lat", "Lng"]]

# Fill NaN values and convert to float
rating = clean_city_data["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [25]:
#Narrow down the DataFrame to find your ideal weather condition.
#cities with temp lower than 85 degress
Temp_data_high = clean_city_data[(clean_city_data["Max Temp"] <80)]
Temp_data_high

#cities with temp greater than 70 degrees
Temp_data_low = Temp_data_high[(Temp_data_high["Max Temp"] >68)]
Temp_data_low

#cities with wind speed less than 10 mph
Wind_Speed_data = Temp_data_low[(Temp_data_low["Wind Speed"] <10)]
Wind_Speed_data

#ciies with cloudiness not equal to zero
my_cities_df = Wind_Speed_data[(Wind_Speed_data["Cloudiness"] ==0)]
my_cities_df 

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
33,Ormara,25.2088,64.6357,77.72,23,0,9.26,PK,1635043335
60,Seoni,22.0833,79.5333,71.53,58,0,2.51,IN,1635043343
144,Beloha,-25.1667,45.0500,68.63,78,0,3.09,MG,1635043367
189,Salalah,17.0151,54.0924,73.49,68,0,5.75,OM,1635043381
238,Santa Rosalía,27.3167,-112.2833,73.67,55,0,6.04,MX,1635043395
360,Capaci,38.1711,13.2393,69.51,93,0,5.75,IT,1635043429
421,Tendūkheda,23.4000,79.5500,72.75,56,0,2.13,IN,1635043447
462,Manaus,-3.1019,-60.0250,79.29,89,0,2.30,BR,1635043459
465,Umluj,25.0213,37.2685,76.14,60,0,9.51,SA,1635043460
531,Beira,-19.8436,34.8389,72.46,83,0,8.25,MZ,1635043479


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [26]:
#Store into variable named hotel_df.
hotel_df = my_cities_df.copy()

#Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"]=""
hotel_df

# find the closest Hotel of each type to coordinates
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    #Set parameters to search for hotels within 5000 meters
    City = f"{hotel_df.loc[index, 'Lat']}, {hotel_df.loc[index, 'Lng']}"
    Vicinity = 5000
    Hotels = "lodging"

    params = {
        "location": City,
        "radius": Vicinity,
        "type": Hotels,
        "key": g_key}

    # assemble url and make API request
    #print the index and the name of the city
    print(f"Retrieving Results for Index {index}: {City}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        #print the index and the closest hotel
        print(f"Retrieving Records for {index}: Closest Hotel is {results[0]['name']}")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")


Retrieving Results for Index 33: 25.2088, 64.6357.
Retrieving Records for 33: Closest Hotel is Crystal Beach and Camping Resort Ormara
------------
Retrieving Results for Index 60: 22.0833, 79.5333.
Retrieving Records for 60: Closest Hotel is STARiHOTELS Nehru Road Seoni
------------
Retrieving Results for Index 144: -25.1667, 45.05.
Missing field/result... skipping.
------------
Retrieving Results for Index 189: 17.0151, 54.0924.
Retrieving Records for 189: Closest Hotel is HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel
------------
Retrieving Results for Index 238: 27.3167, -112.2833.
Retrieving Records for 238: Closest Hotel is Hotel Las Casitas Santa Rosalia
------------
Retrieving Results for Index 360: 38.1711, 13.2393.
Retrieving Records for 360: Closest Hotel is Hotel Sirenetta
------------
Retrieving Results for Index 421: 23.4, 79.55.
Retrieving Records for 421: Closest Hotel is Aditya patel
------------
Retrieving Results for Index 462: -3.1019, -60.025.
Retrieving Records for 462

In [27]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
33,Ormara,25.2088,64.6357,77.72,23,0,9.26,PK,1635043335,Crystal Beach and Camping Resort Ormara
60,Seoni,22.0833,79.5333,71.53,58,0,2.51,IN,1635043343,STARiHOTELS Nehru Road Seoni
144,Beloha,-25.1667,45.0500,68.63,78,0,3.09,MG,1635043367,
189,Salalah,17.0151,54.0924,73.49,68,0,5.75,OM,1635043381,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
238,Santa Rosalía,27.3167,-112.2833,73.67,55,0,6.04,MX,1635043395,Hotel Las Casitas Santa Rosalia
360,Capaci,38.1711,13.2393,69.51,93,0,5.75,IT,1635043429,Hotel Sirenetta
421,Tendūkheda,23.4000,79.5500,72.75,56,0,2.13,IN,1635043447,Aditya patel
462,Manaus,-3.1019,-60.0250,79.29,89,0,2.30,BR,1635043459,Manaus Hotéis Millennium
465,Umluj,25.0213,37.2685,76.14,60,0,9.51,SA,1635043460,Moon light Furnished Units
531,Beira,-19.8436,34.8389,72.46,83,0,8.25,MZ,1635043479,Beira Terrace Hotel


In [28]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [29]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))